# 2. Image Classification with Convolution Neural Network (CNN).

In [79]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

## 1. Load data set

In [80]:
# Step 1 - conver PIL to tensorand normalize data between -1 and 1
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

# Step 2 - load a train set
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

# Step 3 - load a test set
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

## 2. Implement a CNN

In [81]:
# Step 4 - defining a CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # A convolution layer with 32 kernels of size 3×3
        self.conv1 = nn.Conv2d(1, 32, 3)
        # A convolution layer with 64 kernels of size 3×3
        self.conv2 = nn.Conv2d(32, 64, 3)
        # A maxpool layer with kernels of size 2×2.
        self.pool = nn.MaxPool2d(2, 2)
        # A convolution layer with 64 kernels of size 3×3
        self.conv3 = nn.Conv2d(64, 64, 3)
        # A convolution layer with 64 kernels of size 3×3.
        self.conv4 = nn.Conv2d(64, 64, 3)
        # A flattening layer with input size 4096 and output size of 10.
        self.fc1 = nn.Linear(4096, 10)

    def forward(self, x):
        #A ReLU activation for conv1
        x = F.relu(self.conv1(x))
        #A ReLU activation for conv2
        x = self.pool(F.relu(self.conv2(x)))
        #A ReLU activation for max pool & conv3
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 4096)
        x = self.fc1(x)
        return x

net = Net()

## 3. SGD optimizer

In [82]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()
net = net.to(device)

# Step 5 - define a loss funtion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

## 4. Train 10 epochs

In [83]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [84]:
# Step 6 - train network on training set
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

Finished Training


## 5. Predicts labels and classification accuracy. 

In [87]:
# calculate accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %
